# 엑셀 -> 파이썬으로 자동화 처리

In [1]:
"""  0. 데이터 입력  """
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import time
df = pd.read_excel('Data/signal_guide/'+'군포_구급_우선신호_버퍼_교차.xlsx')
# 군포 .. 어느동 어느동 어느동 해서 리스트로 만들어서 For 문 돌려버리면 편할 듯함

In [2]:
df.columns

Index(['no', 'start_date', 'loc', 'div', 'last_date', 'station', 'center',
       'move_station', 'move_center', 'car', 'car_div', 'move_order',
       'move_date', 'field_arrive', 'geocoding', 'x', 'y', 'layer', 'path',
       'date', 'loc_2', 'center_2', 'station_2', 'sec', 'end', 'id',
       'total_time', 'arrive', 'geocoding_2', 'x_2', 'y_2', 'note', 'ringId',
       'distance'],
      dtype='object')

In [237]:
"""  1. Time  """
# move_date 에서 시간만 추출
temp_list = []
for i in range(len(df)):
    temp = df['move_date'][i]
    temp_list.append(str(temp)[11:])

print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['time'] = temp_list

Base Row : 296
Made Row : 296


In [3]:
"""  1. Time _ Lambda """
df['time'] = df['move_date'].apply(lambda x : str(x)[11:])
# ㄷㄷ.. 개쩐다;
# 근데 단순한건 이렇게 람다식으로 할만한데,, 뒤에있는 무친애들은 하기 힘들듯 디버그가 힘들어서

In [4]:
df['time']

0      10:55:47
1      13:04:35
2      17:11:58
3      15:15:37
4      21:23:24
         ...   
291    23:41:26
292    10:00:46
293    11:29:13
294    18:16:20
295    11:14:02
Name: time, Length: 296, dtype: object

In [238]:
"""  2. 시간구분  """
# time 값에 따라 분류
temp_list = []
for i in range(len(df)):
    temp = df['time'][i]
    #print(temp[:2],temp[3:5],temp[6:])
    temp = timedelta(hours=int(temp[:2]), minutes=int(temp[3:5]), seconds=int(temp[6:]))
    #print(temp,type(temp))
    if (temp>=timedelta(hours=11)) & (temp<timedelta(hours=14)):
        temp_list.append("점심")
    elif (temp>=timedelta(hours=17)) & (temp<timedelta(hours=20)):
        temp_list.append("퇴근")
    elif ((temp>=timedelta(hours=20)) & (temp<timedelta(hours=24)) or temp<timedelta(hours=7)):
        temp_list.append("일몰")
    elif (temp>=timedelta(hours=7)) & (temp<timedelta(hours=10)):
        temp_list.append("출근")
    else:
        temp_list.append("낮")
    
print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['시간구분'] = temp_list
    


Base Row : 296
Made Row : 296


In [8]:
"""  3. Total_time  """
from datetime import datetime
import time
# field_arrive - move_date
temp_list = []
for i in range(len(df)):
    #temp = time.mktime(datetime.strptime(df['field_arrive'][i][:-4], '%Y-%m-%dT%H:%M:%S').timetuple())
    #print(df['field_arrive'][i])
    temp = pd.Timestamp(df['field_arrive'][i]) # 앞쪽 값에만 .000 붙어있어서 일괄적으로 처리할시 오류생김 -> .으로 Split 하고, [0]으로 맨 앞 값 선택하면 해결될 것 -> 또 오류
    # 뒤쪽은 날짜 구분자가 . 으로 되어있어서 오류발생.. -> ??? 굳이 .000 없앨 필요가 없었음
    #print(temp)#,type(temp))
    #print(df['move_date'][i])#,type(df['move_date'][i]))
    temp_list.append(temp - df['move_date'][i])
    #print(temp)

print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['total_time2'] = temp_list

Base Row : 296
Made Row : 296


In [13]:
pd.Timestamp(df['field_arrive'][1])

Timestamp('2019-01-06 13:08:44')

In [26]:
df['field_arrive'].apply(lambda x:x[:19])

0      2019-01-04T10:57:32
1      2019-01-06T13:08:44
2      2019-01-07T17:21:58
3      2019-01-13T15:18:39
4      2019-01-14T21:26:14
              ...         
291    2021.12.22 23:43:23
292    2021.12.23 10:07:18
293    2021.12.24 11:42:58
294    2021.12.28 18:19:45
295    2021.12.29 11:20:04
Name: field_arrive, Length: 296, dtype: object

In [18]:
df['total_time2'] = (lambda x,y : pd.Timestamp(x) - y)(df['field_arrive'],df['move_date'])

TypeError: Cannot convert input [0      2019-01-04T10:57:32.000
1      2019-01-06T13:08:44.000
2      2019-01-07T17:21:58.000
3      2019-01-13T15:18:39.000
4      2019-01-14T21:26:14.000
                ...           
291        2021.12.22 23:43:23
292        2021.12.23 10:07:18
293        2021.12.24 11:42:58
294        2021.12.28 18:19:45
295        2021.12.29 11:20:04
Name: field_arrive, Length: 296, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp

In [15]:
df['total_time2'] = df['field_arrive'].apply(lambda x,y : pd.Timestamp(x) - y)(df['move_date'])

TypeError: <lambda>() missing 1 required positional argument: 'y'

In [17]:
df.apply(lambda x : pd.Timestamp(x['field_arrive']) - x['move_date'])

KeyError: 'field_arrive'

In [240]:
"""  4. time2  """
# date 에서 시간 추출 + date 형태 변환
import datetime
temp_list = []
temb_list = []
for i in range(len(df)):
    temp = (datetime.datetime.strptime('1899-12-30', '%Y-%m-%d') + datetime.timedelta(df['date'][i])).strftime('%Y-%m-%d %H:%M:%S')
    temp_list.append(temp)
    temb_list.append(str(temp)[11:])
    
print("Base Row :",len(df))
print("Made Row_1 :",len(temp_list))
print("Made Row_2 :",len(temb_list))
df['date'] = temp_list
df['time2'] = temb_list

Base Row : 296
Made Row_1 : 296
Made Row_2 : 296


In [241]:
""" 5. 시간구분2  """
# time2 값에 따라 분류
temp_list = []
for i in range(len(df)):
    temp = df['time2'][i]
    #print(temp[:2],temp[3:5],temp[6:])
    temp = timedelta(hours=int(temp[:2]), minutes=int(temp[3:5]), seconds=int(temp[6:]))
    #print(temp,type(temp))
    if (temp>=timedelta(hours=11)) & (temp<timedelta(hours=14)):
        temp_list.append("점심")
    elif (temp>=timedelta(hours=17)) & (temp<timedelta(hours=20)):
        temp_list.append("퇴근")
    elif ((temp>=timedelta(hours=20)) & (temp<timedelta(hours=24)) or temp<timedelta(hours=7)):
        temp_list.append("일몰")
    elif (temp>=timedelta(hours=7)) & (temp<timedelta(hours=10)):
        temp_list.append("출근")
    else:
        temp_list.append("낮")
    
print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['시간구분2'] = temp_list

Base Row : 296
Made Row : 296


In [244]:
"""  6. 소요시간  """
# total_time2 - total_time
temp_list = []
for i in range(len(df)):
    temp = datetime.timedelta(df['total_time'][i])
    temp_list.append(df['total_time2'][i] - temp)

print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['소요시간'] = temp_list

Base Row : 296
Made Row : 296


In [5]:
""" 6. 소요시간 _ lambda"""
df[]

''

In [245]:
"""  7. 동일시간  """
# 시간구분 == 시간구분2
temp_list = []
for i in range(len(df)):
    if df['시간구분'][i] == df['시간구분2'][i]:
        temp_list.append("TRUE")
    else:
        temp_list.append("FALSE")
print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['동일시간'] = temp_list


Base Row : 296
Made Row : 296


### 테이블 간 컬럼 차이
- 군포 : car_2 없음
- 화성 : center_2 없음
- 해결방법 : 지역 2 데이터 처리시 Try - Except 처리

In [246]:
"""  8. 지역  """ # 지역, 지역 2,
temp_list = []
for i in range(len(df)):
    #temp = df['center'][i][:2]
    #print(df['center'][i],temp)
    temp_list.append(df['center'][i][:2])

print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['지역'] = temp_list

Base Row : 296
Made Row : 296


In [247]:
"""  9. 지역2  """
temp_list = []
for i in range(len(df)):
    try: # 군포, 파주 테이블 처리 (Center_2 있음)
        temp = df['center_2'][i][:2]
        #print(df['center_2'][i],temp,"1")
    except: # 화성 테이블 처리 (Center_2 없음)
        temp = df['car_2'][i][:2]
        #print(df['center_2'][i],temp,"2")
        
    temp_list.append(temp)

print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['지역2'] = temp_list

Base Row : 296
Made Row : 296


In [248]:
"""  10. 동일지역  """
temp_list = []
for i in range(len(df)):
    if df['지역'][i] == df['지역2'][i]:
        temp_list.append("TRUE")
    else:
        temp_list.append("FALSE")

print("Base Row :",len(df))
print("Made Row :",len(temp_list))
df['동일지역'] = temp_list

Base Row : 296
Made Row : 296


In [251]:
"""  11. 결과출력  """
df = df.set_index('no')
with pd.ExcelWriter('Data/signal_guide/군포_구급_우선신호_버퍼_교차_시간차.xlsx') as writer:
    df.to_excel(writer, sheet_name = '군포_구급_우선신호_버퍼_교차')
    df[df['동일시간']=="TRUE"].to_excel(writer, sheet_name = '동일시간')
    df[df['동일지역']=="TRUE"].to_excel(writer, sheet_name = '동일지역')

### 주의사항
- 앞, 뒤 분류가 애매한 녀석들 있음
- 저거 분류 이후에도 있음
- 한 두시간 걸릴라나? 오늘안에 Ipynb 마무리

### 문의사항
- 각 시트별 컬럼, 컬럼위치 맞춰야 하는지? : 아니용

### .py 만들때 유의점
- for 문으로 리스트로 준 지역명 전체 돌리도록